In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ./darts_logs --host localhost --port 8099

In [2]:
import os, sys

parent = os.path.abspath('..')
sys.path.insert(1, parent)

In [3]:
from helper_funcs.preprocessing import total_timeseries, get_covariates
from helper_funcs.prediction import historical_predictions, total_validation
from helper_funcs.inverse import inverse_func

from darts.utils.model_selection import train_test_split
from darts.dataprocessing.transformers import StaticCovariatesTransformer, Scaler
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar
from darts.models import NLinearModel, TFTModel, NaiveSeasonal, NHiTSModel

import torch
torch.set_float32_matmul_precision('medium')
import eco2ai

from models import nlinear, tft, nhits

RANDOM = 101
INPUT_CHUNK = 15
OUTPUT_CHUNK = 3
TEST_SIZE = 0.2
RETRAIN=True
LAST=False
RESET=False
EXP_MA = 15
SAVE = True
EPOCHS = 100

checkpoint = ModelCheckpoint(monitor="val_loss")
progress_bar = RichProgressBar()
tracker = eco2ai.Tracker(project_name="Stock_Market_Prediction", experiment_description="training multiple models")

/home/andrei/Documents/HSE_Masters/HSE_Masters/tft_env/lib/python3.10/site-packages/eco2ai/emission_track.py:135: UserWarning: 
If you use a VPN, you may have problems with identifying your country by IP.
It is recommended to disable VPN or
manually install the ISO-Alpha-2 code of your country during initialization of the Tracker() class.
You can find the ISO-Alpha-2 code of your country here: https://www.iban.com/country-codes

  warnings.warn(


In [4]:
callbacks=[progress_bar]

In [5]:
timeseries = total_timeseries(EXP_MA, market=True, sentiment=True, embeddings=True, large=True)
timeseries = StaticCovariatesTransformer().fit_transform(timeseries)

In [6]:
# for i in range(6):
#     timeseries[i].to_pickle(f'./ts_pickle/timeseries_{i}.pkl')

In [7]:
train, val = train_test_split(
    timeseries,
    axis=1,
    test_size=TEST_SIZE,
    input_size=INPUT_CHUNK,
    horizon=OUTPUT_CHUNK,
    vertical_split_type='model-aware'
)

data = [train, val]

target_train, past_train, future_train, target_val, past_val, future_val, target_test, past_test, future_test = get_covariates(
    data,
    target='Close',
    past_covariates=[
        # 'Adj Close',
        'High',
        'Low',
        'Open',
        'Volume',
        # 'Volatility',
        # 'Negative',
        # 'Positive',
        # 'sentiment_score_1',
        # 'sentiment_score_2'
        ],
    embeddings=True
    )

name = 'model_TFT_HLOVE'

In [8]:
past_train[0].components

Index(['High', 'Low', 'Open', 'Volume', '0', '1', '2', '3', '4', '5',
       ...
       '758', '759', '760', '761', '762', '763', '764', '765', '766', '767'],
      dtype='object', name='component', length=772)

In [9]:
scaler_target = Scaler()
scaler_covariates = Scaler()

In [10]:
target_train_scaled = scaler_target.fit_transform(target_train)
target_val_scaled = scaler_target.transform(target_val)
target_test_scaled = scaler_target.transform(target_test)


past_train_scaled = scaler_covariates.fit_transform(past_train)
past_val_scaled = scaler_covariates.transform(past_val)
past_test_scaled = scaler_covariates.transform(past_test)

In [11]:
# baseline = NaiveSeasonal(K=5)

# baseline_hist = historical_predictions(
#     baseline, target_val_scaled, INPUT_CHUNK, OUTPUT_CHUNK, RETRAIN, LAST, \
#         save=SAVE, past=past_val_scaled, future=future_val)

In [ ]:
# tracker.start()

# if name.split('_')[1] == 'nlinear':
#     model = nlinear.nlinear_emb(name, INPUT_CHUNK, OUTPUT_CHUNK, RANDOM, \
#         callbacks, target_train_scaled, target_val_scaled, past_train_scaled, past_val_scaled, \
#             future_train, future_val, EPOCHS)
            
#     model = NLinearModel.load_from_checkpoint(name, best=True, map_location=torch.device("cuda:0"))

# if name.split('_')[1] == 'tft':
#     model = tft.tft_set(name, INPUT_CHUNK, OUTPUT_CHUNK, RANDOM, \
#         callbacks, target_train_scaled, target_val_scaled, past_train_scaled, past_val_scaled, \
#             future_train, future_val, EPOCHS)
    
#     model = TFTModel.load_from_checkpoint(name, best=True, map_location=torch.device("cuda:0"))

# if name.split('_')[1] == 'nhits':
#     model = nhits.nhits(name, INPUT_CHUNK, OUTPUT_CHUNK, RANDOM, \
#         callbacks, target_train_scaled, target_val_scaled, past_train_scaled, past_val_scaled, \
#             future_train, future_val, EPOCHS)
            
#     model = NHiTSModel.load_from_checkpoint(name, best=True, map_location=torch.device("cuda:0"))

In [12]:
# model = tft.tft_set(name, INPUT_CHUNK, OUTPUT_CHUNK, RANDOM, \
#         callbacks, target_train_scaled, target_val_scaled, past_train_scaled, past_val_scaled, \
#             future_train, future_val, EPOCHS)
    
model = TFTModel.load_from_checkpoint(name, best=True, map_location=torch.device("cuda:0"))

In [13]:
model.model_params

OrderedDict([('hidden_size', 64),
             ('lstm_layers', 1),
             ('num_attention_heads', 4),
             ('full_attention', True),
             ('feed_forward', 'SwiGLU'),
             ('dropout', 0.25),
             ('hidden_continuous_size', 32),
             ('categorical_embedding_sizes', None),
             ('add_relative_index', False),
             ('loss_fn', DirectionalLossAll()),
             ('likelihood', None),
             ('norm_type', 'RMSNorm'),
             ('input_chunk_length', 15),
             ('output_chunk_length', 3),
             ('torch_metrics', MeanSquaredError()),
             ('optimizer_cls', torch.optim.adam.Adam),
             ('optimizer_kwargs', {'lr': 3e-05}),
             ('lr_scheduler_cls', torch.optim.lr_scheduler.StepLR),
             ('lr_scheduler_kwargs', {'step_size': 20, 'gamma': 0.5}),
             ('batch_size', 32),
             ('n_epochs', 100),
             ('model_name', 'model_TFT_HLOVE'),
             ('log_tensorb

In [ ]:
# model.model_params['n_epochs'] = 100
# model.model_params['n_epochs']

In [ ]:
model.model_params['model_name'] = name
model.model_params['model_name']

In [14]:
model.epochs_trained

0

In [ ]:
import pytorch_lightning as pl

trainer_params = model.trainer_params
trainer_params.update({
    'logger': False,
    'enable_model_summary': False,
    'enable_progress_bar': False
})
trainer_params['callbacks'] = [
    c for c in trainer_params['callbacks'] if 
                               ('TQDMProgressBar' not in str(c.__class__))
]

model.trainer = pl.Trainer(**trainer_params)
model.model.trainer = model.trainer

In [ ]:
# model.model_params['force_reset'] = True

In [15]:
reduced_val =[]
reduced_past = []
reduced_future = []

for i, j, k in zip(target_val_scaled, past_val_scaled, future_val):
    reduced_val.append(i[:100])
    reduced_past.append(j[:100])
    reduced_future.append(k[:100])

In [ ]:
hist = historical_predictions(
    model, reduced_val, INPUT_CHUNK, OUTPUT_CHUNK, RETRAIN, LAST, \
        save=SAVE, past=reduced_past, future=reduced_future)

# tracker.stop()